<br>

<img src="https://fedlex.data.admin.ch/assets/logo.png" style="width:25%; float:right">

# Fedlex Tutorial

This page serves as a introductory tutorial to the [Fedlex Linked Data ecosystem](https://fedlex.data.admin.ch).



# Introduction

The webpage you are currently viewing is a so called **interactive JupyterLite notebook**. In this notebook, you can change interactively the content of the single cells and execute these cells directly seeing the result of your changes immediately. The cells contain either [Markdown](https://en.wikipedia.org/wiki/Markdown) content (like this cell) or executable Python source code.

JupyterLite stems from JupyterLab with the advantage of being completely browser based without any backend infrastructure. This means that the execution of the cells could take some time during first execution. Subsequent executions will be much faster because of stored data in your browser cache.

If you are unfamiliar with the handling of Jupyter notebooks, here are two useful resources:

- [The JupyterLab Interface](https://jupyterlab.readthedocs.io/en/stable/user/interface.html)
- [The Jupyter Notebook](https://jupyterlab.readthedocs.io/en/stable/user/notebook.html)

# Preliminaries

In this part some preliminaries for querying LINDAS with SPARQL are introduced. If you are only interested in the actual LINDAS tutorial, you can skip the whole section and start [here](#Actual-Tutorial).

## Install and Import the Necessary Modules

Querying a SPARQL endpoint is basically making a POST request to the corresponding endpoint URL. As JupyterLite at the moment has no support for Python's `requests` modul, the JavaScript fetch API is used (with some tricks). To make this happen, the following modules have to be importet: 

In [83]:
%pip install -q folium

In [84]:
import json
import pandas as pd
from pyodide.ffi import to_js
from IPython.display import JSON, HTML
from js import Object, fetch
from io import StringIO

## Define Main Query Function

As the JavaScript fetch API is asynchronous, the corresponding Python function `query` has to be declared as `async`. This function allows to query the 3 Swiss governmental triple stores.

In [85]:
async def query(query_string, store = "L"):
    
    # three Swiss triplestores
    if store == "F":
        address = 'https://fedlex.data.admin.ch/sparqlendpoint'
    elif store == "G":
        address = 'https://geo.ld.admin.ch/query'
    else:
        address = 'https://ld.admin.ch/query'
    
    # try the Post request with help of JS fetch
    # the creation of the request header is a little bit complicated because it needs to be a 
    # JavaScript JSON that is made within a Python source code
    try:
        resp = await fetch(address,
          method="POST",
          body="query=" + query_string,
          credentials="same-origin",
          headers=Object.fromEntries(to_js({"Content-Type": "application/x-www-form-urlencoded; charset=UTF-8", 
                                            "Accept": "text/csv" })),
        )
    except:
        raise RuntimeError("fetch failed")
    
    
    if resp.ok:
        res = await resp.text()
        # ld.admin.ch throws errors starting with '{"message":'
        if '{"message":' in res:
            error = json.loads(res)
            raise RuntimeError("SPARQL query malformed: " + error["message"])
        # geo.ld.admin.ch throws errors starting with 'Parse error:'
        elif 'Parse error:' in res:
            raise RuntimeError("SPARQL query malformed: " + res)
        else:
            # if everything works out, create a pandas dataframe from the csv result
            df = pd.read_csv(StringIO(res))
            return df
    else:
        # fedlex.data.admin.ch throws error with response status 400
        if resp.status == 400:
            raise RuntimeError("Response status 400: Possible malformed SPARQL query. No syntactic advice available.")
        else:
            raise RuntimeError("Response status " + resp.status)

If you are interested in the details of using the JavaScript fetch API within JupyterLite, please consult:

- https://pyodide.org/en/stable/usage/faq.html#how-can-i-use-fetch-with-optional-arguments-from-python
- https://github.com/jupyterlite/jupyterlite/discussions/412
- https://lwebapp.com/en/post/pyodide-fetch

## Define Display Function

Displays pandas dataframe resulting from the SPARQL query as HTML with clickable links.

In [86]:
def display_result(df):
    df = HTML(df.to_html(render_links=True, escape=False))
    display(df)

# Tutorial

## Vokabular

### ConceptScheme

Ein guter Weg um mit den Fedlex Daten vertraut zu werden ist, sich das verwendete Vokabular genauer anzuschauen. Das Vokabular ist mit Hilfe von [skos:ConceptScheme](http://www.w3.org/2004/02/skos/core#ConceptScheme) zusammengefasst und tritt in Fedlex typischerweise auf der Objekt-Position innerhalb eines Triples auf. Es kann also mit Hilfe von SPARQL nach solchen *ConceptSchemes* gesucht werden:

In [87]:
df = await query("""

PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX purl: <http://purl.org/dc/terms/>

SELECT DISTINCT * WHERE {
    ?conceptScheme a skos:ConceptScheme;
        purl:title ?title.
FILTER(lang(?title) = "de")
} 

""", "F")

display_result(df)

,conceptScheme,title
0,https://fedlex.data.admin.ch/vocabulary/consultation-stage,Arbeitsschritte eines Vernehmlassungsverfahrens
1,https://fedlex.data.admin.ch/vocabulary/consultation-status,Stand des Vernehmlassungsverfahrens
2,https://fedlex.data.admin.ch/vocabulary/country,Staaten
3,https://fedlex.data.admin.ch/vocabulary/eutext-subject-theme,Klassifizierung der sektoriellen Abkommen mit der Europäischen Union
4,https://fedlex.data.admin.ch/vocabulary/impact-type,Betroffenheiten
5,https://fedlex.data.admin.ch/vocabulary/information-source,Art der Datenquelle
6,https://fedlex.data.admin.ch/vocabulary/instrument-type,Art des Rechtsaktes
7,https://fedlex.data.admin.ch/vocabulary/international-actor,Internationale Organisationen
8,https://fedlex.data.admin.ch/vocabulary/legal-analysis-status,Stand der Erfassung der Betroffenheiten
9,https://fedlex.data.admin.ch/vocabulary/legal-institution,Organisationseinheiten


### Inhalt eines einzelnen ConceptSchemes

Wenn die Inhalte eines einzelnen ConecptSchemes abgefragt werden soll, kann folgende SPARQL Query helfen:

In [88]:
df = await query("""

PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX purl: <http://purl.org/dc/terms/>

SELECT DISTINCT * WHERE {
    ?concept skos:inScheme <https://fedlex.data.admin.ch/vocabulary/treaty-status>;
        skos:prefLabel ?label.
    FILTER(lang(?label) = "de")
} 

""", "F")

display_result(df)

,concept,label
0,https://fedlex.data.admin.ch/vocabulary/treaty-status/1,abgeschlossen (drafts)
1,https://fedlex.data.admin.ch/vocabulary/treaty-status/2,in Kraft
2,https://fedlex.data.admin.ch/vocabulary/treaty-status/3,unterzeichnet
3,https://fedlex.data.admin.ch/vocabulary/treaty-status/4,provisorische Anwendung
4,https://fedlex.data.admin.ch/vocabulary/treaty-status/5,abgelaufen
5,https://fedlex.data.admin.ch/vocabulary/treaty-status/6,ratifiziert


### Dereferenziertes Vokabular

Das gesamte Vokabular kann auch über https://fedlex.data.admin.ch/vocabularies direkt als Webpage abgerufen werden. Diese Website ist hierarchisch aufgebaut und wenn auf https://fedlex.data.admin.ch/vocabulary/treaty-status geklickt wird, erhält man alle Infos zu diesem spezifischen skos:ConceptScheme als Website. Analog gilt dies auch für die einzelnen skos:Concept (bspw. https://fedlex.data.admin.ch/vocabulary/treaty-status/1).

## Aufbau der Daten

### URI Schema

Die eigentlichen Daten (nämlich Gesetzestexte) sind nach einem Schema mit URI versehen. Dieses [URI Schema]https://fedlex.data.admin.ch/de-CH/home/convention richtet sich nach dem System der [European Legislation Identifier (ELI)](https://eur-lex.europa.eu/eli-register/about.html).

Grundsätzlich beginnen alle Daten mit folgender URI: http://fedlex.data.admin.ch/eli

### Sammlungen

* Bundesblatt ([Erläuterungen](https://www.fedlex.admin.ch/de/fga/explanation-fg))
* Amtliche Sammlung ([Erläuterungen](https://www.fedlex.admin.ch/de/oc/explanations-oc))
* Systematische Sammlung ([Erläuterungen](https://www.fedlex.admin.ch/de/cc/explanations-cc))

### Bundesblatt

Folgende Query gibt die aktuelle Ausgabe des Bundesblatts zurück:

In [89]:
df = await query("""

PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX purl: <http://purl.org/dc/terms/>
PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>

SELECT DISTINCT * WHERE {
    ?fga jolux:isMemberOf* <https://fedlex.data.admin.ch/eli/collection/fga>;
        jolux:publicationDate ?date.
} ORDER BY desc(?date) LIMIT 1

""", "F")

display_result(df)

,fga,date
0,https://fedlex.data.admin.ch/eli/collection/fga/2023/30,2023-02-14


### Amtliche Sammlung

In einem weiteren Schritt geht es darum, den Aufbau der Daten unter https://fedlex.data.admin.ch besser zu verstehen. Ein guter Einstiegspunkt ist die Suche unter https://fedlex.admin.ch. Wenn wir dort in der systematischen Rechtssammlung nach der Bundesverfassung suchen, gelangen wir auf die Seite https://www.fedlex.admin.ch/eli/oc/1999/404. Das ist auch gerade die URI der Bundesverfassung.

Wir können nun eine SPARQL Query schreiben, mit der wir danach suchen wollen, wo die Bundesverfassung überall auf der Subjekt-Position vorkommt, um weiter ins Datenmodell einzutauchen. Wir fragen also danach, welche Metadaten zur Bundesverfassung gespeichert sind:

In [90]:
df = await query("""

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT *  WHERE {
  <https://fedlex.data.admin.ch/eli/oc/1999/404> ?p ?o.
} 

""", "F")

display_result(df)

,p,o
0,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://data.legilux.public.lu/resource/ontology/jolux#Work
1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://data.legilux.public.lu/resource/ontology/jolux#WorkAtOj
2,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://data.legilux.public.lu/resource/ontology/jolux#Act
3,http://data.legilux.public.lu/resource/ontology/jolux#legalResourceGenre,https://fedlex.data.admin.ch/vocabulary/legal-resource-genre/100
4,http://data.legilux.public.lu/resource/ontology/jolux#processType,https://fedlex.data.admin.ch/vocabulary/type-procedure/ord
5,http://data.legilux.public.lu/resource/ontology/jolux#responsibilityOf,https://fedlex.data.admin.ch/vocabulary/legal-institution/5
6,http://data.legilux.public.lu/resource/ontology/jolux#sequenceInTheYearOfPublication,404
7,http://data.legilux.public.lu/resource/ontology/jolux#typeDocument,https://fedlex.data.admin.ch/vocabulary/resource-type/55
8,http://data.legilux.public.lu/resource/ontology/jolux#isPartOf,https://fedlex.data.admin.ch/eli/collection/oc/1999/42
9,http://data.legilux.public.lu/resource/ontology/jolux#publicationDate,1999-10-26
